### Метод рунге-кутты 4-го порялка

In [1]:

def rungeKutta2Step(f, g, x, y, v, xstep):
    k1 = xstep * f(x, y, v)
    l1 = xstep * g(x, y, v)

    k2 = xstep * f(x + xstep / 2, y + k1 / 2, v + l1 / 2)
    l2 = xstep * g(x + xstep / 2, y + k1 / 2, v + l1 / 2)

    nexty = y + (k1 + k2) / 2
    nextv = v + (l1 + l2) / 2
    return nexty, nextv


def rungeKutta2(x0, u0, v0, xstep, xTarget, f, g):
    x, u, v = x0, u0, v0
    result = [(x, u, v)]
    while x <= xTarget:
        u, v = rungeKutta2Step(f, g, x, u, v, xstep)
        result.append((x, u, v))
        x += xstep
    return result


def rungeKutta4Step(f, g, x, u, v, xstep):
    k1 = xstep * f(x, u, v)
    l1 = xstep * g(x, u, v)

    k2 = xstep * (f(x + 0.5 * xstep, u + 0.5 * k1, v + 0.5 * l1))
    l2 = xstep * (g(x + 0.5 * xstep, u + 0.5 * k1, v + 0.5 * l1))

    k3 = xstep * (f(x + 0.5 * xstep, u + 0.5 * k2, v + 0.5 * l2))
    l3 = xstep * (g(x + 0.5 * xstep, u + 0.5 * k2, v + 0.5 * l2))

    k4 = xstep * (f(x + xstep, u + k3, v + l3))
    l4 = xstep * (g(x + xstep, u + k3, v + l3))

    nextu = u + (k1 + 2 * k2 + 2 * k3 + k4) / 6
    nextv = v + (l1 + 2 * l2 + 2 * l3 + l4) / 6
    return nextu, nextv

def rungeKutta4(x0, u0, v0, xstep, xTarget, f, g):
    x, u, v = x0, u0, v0
    result = [(x, u, v)]
    while x <= xTarget:
        u, v = rungeKutta4Step(f, g, x, u, v, xstep)
        result.append((x, u, v))
        x += xstep
    return result

### Интерполяция плотности плазмы

In [2]:
import numpy as np

TempTable = ((2000, 8.200E-03),
    (3000, 2.768E-02),
    (4000, 6.560E-02),
    (5000, 1.281E-01),
    (6000, 2.214E-01),
    (7000, 3.516E-01),
    (8000, 5.248E-01),
    (9000, 7.472E-01),
    (10000, 1.025E+00),
)



# TempTable = ((2000, 1.6),
#     (3000, 5.4),
#     (4000, 1.280E+01),
#     (5000, 2.500E+01),
#     (6000, 4.320E+01),
#     (7000, 6.860E+01),
#     (8000, 1.024E+02),
#     (9000, 1.458E+02),
#     (10000, 2.000E+02),
# )

def getkFunc():
    tarr, karr = zip(*TempTable)
    tarr = [np.log(x) for x in tarr]
    karr = [np.log(x) for x in karr]
    def k(t):
        return np.exp(np.interp([np.log(t)], tarr, karr)[0])
    return k


### Формула планка и параметры 

In [3]:
Tw = 2000
T0 = 10000
R = 0.35
p = 4

c = 3 * 1e10 # Скорость света

def T(r):
    return (Tw - T0) * (r / R)**p + T0

#u_p(r)
def plankFunc(r):
    return 3.084 * 1e-4 / (np.exp(4.799*1e4 / T(r)) - 1)


### Вычисление варьируемого параметра

In [4]:
# u(0) = lambda * u_p(0)
# v(0) = 0
# v(R) = 0.39 * c * u(R)

k = getkFunc()

def f(x, u, v):
    return (-3) * k(T(x)) * v / c

def g(x, u, v):
    if (x == 0):
        return k(T(x)) * c * (plankFunc(x) - u) / 2
    return k(T(x)) * c * (plankFunc(x) - u) - v / x

def findSolution(lambdaParam, rungeKuttaEps, initStep):
    x0 = 0
    u0 = lambdaParam * plankFunc(0)
    v0 = 0
    xstep = initStep
    xTarget = R
    prevResult = rungeKutta4(x0, u0, v0, xstep, xTarget, f, g)
    prevTargetu = prevResult[-1][1]
    prevTargetv = prevResult[-1][2]
    while True:
        xstep /= 2
        result = rungeKutta4(x0, u0, v0, xstep, xTarget, f, g)
        targetu = result[-1][1]
        targetv = result[-1][2]
        if abs(prevTargetu - targetu) / targetu  < rungeKuttaEps:
            break
        prevResult = result
        prevTargetu = targetu
        prevTargetv = targetv
    return targetu, targetv, xstep

def findPoints(lambdaParam, rungeKuttaEps, initStep):
    x0 = 0
    u0 = lambdaParam * plankFunc(0)
    v0 = 0
    xstep = initStep
    xTarget = R
    prevResult = rungeKutta4(x0, u0, v0, xstep, xTarget, f, g)
    prevTargetu = prevResult[-1][1]
    prevTargetv = prevResult[-1][2]
    while True:
        xstep /= 2
        result = rungeKutta4(x0, u0, v0, xstep, xTarget, f, g)
        targetu = result[-1][1]
        targetv = result[-1][2]
        if abs(prevTargetu - targetu) / targetu  < rungeKuttaEps:
            break
        prevResult = result
        prevTargetu = targetu
        prevTargetv = targetv
    return result

def equationFunc(x, u, v):
    return 0.39 * c * u - v

def findLambda(eps, rungeKuttaEps, verbose=False):
    left = 0.1
    leftResult = findSolution(left, rungeKuttaEps, 1e-2)
    right = 1
    rightResult = findSolution(right, rungeKuttaEps, 1e-2)

    

    # Метод половинного деления
    while abs((right - left) / left) > eps:
        mid = (left + right) / 2
        midResult = findSolution(mid, rungeKuttaEps, 1e-2)
        if equationFunc(mid, midResult[0], midResult[1]) * equationFunc(left, leftResult[0], leftResult[1]) < 0:
            right = mid
        else:
            left = mid
        if verbose:
            print(f"lambda = {mid}, u = {midResult[0]}, v = {midResult[1]}, step = {midResult[2]}")
    if verbose:
        print(f"final lambda = {right}, u = {rightResult[0]}, v = {rightResult[1]}, step = {midResult[2]}")
    return right


lambdaParam = findLambda(eps = 1e-10, rungeKuttaEps=1e-4, verbose=True)

lambda = 0.55, u = 1.3689071646221557e-06, v = 746.9266944064335, step = 0.005
lambda = 0.325, u = 7.664919990836098e-07, v = 2369.1708157849885, step = 0.005
lambda = 0.21250000000000002, u = 4.6528441631433644e-07, v = 3180.292876474266, step = 0.005
lambda = 0.15625, u = 3.146806249296999e-07, v = 3585.853906818904, step = 0.005
lambda = 0.128125, u = 2.393787292373815e-07, v = 3788.634421991224, step = 0.005
lambda = 0.1421875, u = 2.770296770835408e-07, v = 3687.2441644050646, step = 0.005
lambda = 0.14921875, u = 2.9585515100662056e-07, v = 3636.5490356119853, step = 0.005
lambda = 0.152734375, u = 3.052678879681601e-07, v = 3611.2014712154446, step = 0.005
lambda = 0.1544921875, u = 3.0997425644893e-07, v = 3598.5276890171754, step = 0.005
lambda = 0.15361328125, u = 3.0762107220854536e-07, v = 3604.8645801163107, step = 0.005
lambda = 0.154052734375, u = 3.0879766432873757e-07, v = 3601.696134566742, step = 0.005
lambda = 0.1538330078125, u = 3.0820936826864154e-07, v = 3603.28

### Решение

In [5]:
import plotly.express as px
import plotly.graph_objects as go

eps = 1e-4
print(f"Lambda найдена: {lambdaParam}+-{lambdaParam*eps}")


# rstep = R / rNumber
# rvalues, uvalues, vvalues = zip(*rungeKutta4(0, lambdaParam * plankFunc(0), 0, rstep, R, f, g))
rvalues, uvalues, vvalues = zip(*findPoints(lambdaParam, 1e-4, 1e-2))
u_pvalues = [plankFunc(r) for r in rvalues]

fig = px.line()
fig.add_trace(go.Scatter(x=rvalues, y=uvalues, name="u"))
fig.add_trace(go.Scatter(x=rvalues, y=u_pvalues, name="u_p"))
# fig.update_layout(yaxis_type = "log")
fig.show()

fig = px.line()
fig.add_trace(go.Scatter(x=rvalues, y=vvalues, name="F"))
fig.show()

Lambda найдена: 0.15376114559185228+-1.5376114559185227e-05


In [6]:
Rs = [0.35, 0.5, 0.7, 1, 1.5, 2, 3]

rNumber = 1000


prevR = R
for r in Rs:
    fig = px.line()
    R = r
    lambdaParam = findLambda(1e-4, 1e-4)
    print(f"Rs: {R}, lambda: {lambdaParam}")
    rvalues, uvalues, vvalues = zip(*findPoints(lambdaParam, 1e-4, 1e-2))
    fig.add_trace(go.Scatter(x=rvalues, y=uvalues, name=f"u, R={R}"))
    rvalues, _, _ = zip(*findPoints(lambdaParam, 1e-4, 1e-2))
    u_pvalues = [plankFunc(r) for r in rvalues]
    fig.add_trace(go.Scatter(x=rvalues, y=u_pvalues, name=f"u_p, R={R}"))
    fig.show()
    

R = prevR

    


Rs: 0.35, lambda: 0.15376434326171876


Rs: 0.5, lambda: 0.21883087158203124


Rs: 0.7, lambda: 0.302642822265625


Rs: 1, lambda: 0.4190704345703125


Rs: 1.5, lambda: 0.58197021484375


Rs: 2, lambda: 0.70545654296875


Rs: 3, lambda: 0.857232666015625


In [7]:
TempTableMultipliers = [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20]

fig = px.line()
prevTempTable = TempTable
for mult in TempTableMultipliers:
    TempTable = [(elem[0], elem[1] * mult) for elem in TempTable]
    k = getkFunc()
    lambdaParam = findLambda(1e-4, 1e-4)
    rvalues, uvalues, vvalues = zip(*findPoints(lambdaParam, 1e-4, 1e-2))
    fig.add_trace(go.Scatter(x=rvalues, y=uvalues, name=f"u, kmult={mult}"))
    TempTable = prevTempTable
k = getkFunc()
rvalues, _, _ = zip(*findPoints(lambdaParam, 1e-4, 1e-2))
u_pvalues = [plankFunc(r) for r in rvalues]
fig.add_trace(go.Scatter(x=rvalues, y=u_pvalues, name=f"u_p"))

fig.show()



In [8]:
pValues = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20]

fig = px.line()

prevP = p
for p in pValues:
    p = p
    lambdaParam = findLambda(1e-4, 1e-4)
    rvalues, uvalues, vvalues = zip(*findPoints(lambdaParam, 1e-4, 1e-2))
    fig.add_trace(go.Scatter(x=rvalues, y=uvalues, name=f"u, p={p}"))

rvalues, _, _ = zip(*findPoints(lambdaParam, 1e-4, 1e-2))
u_pvalues = [plankFunc(r) for r in rvalues]
fig.add_trace(go.Scatter(x=rvalues, y=u_pvalues, name=f"u_p"))

fig.show()
        